In [1]:
import os
from dotenv import load_dotenv

In [3]:
try:
    load_dotenv()
    print("Key successfully imported")
except Exception as e:
    print("failed to load the env file")

Key successfully imported


In [6]:
try:
    GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("Authentication successful for the key")
except Exception as e:
    print(f"Error loading the key {e}")

Authentication successful for the key


# Importing Google's ADK components


In [7]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("all imports were successful")

all imports were successful


# Retry options

if the process fails to run, a backup which would retry on it's own

In [11]:
retry_config = types.HttpRetryOptions(
    attempts = 5,
    exp_base = 7,
    initial_delay = 1,
    http_status_codes = [429,500,503,504],
)
print("retry config working")

retry config working


## Section - Why Multi-Agent Systems? + Your First multi-agent

### Why we need multi agents - "the do it all agent"

Single agents can do a lot. But what happens when the task gets complex? A single "monolithic" agent that tries to do research, writing, editing, 
and fact-checking all at once becomes a problem. Its instruction prompt gets long and confusing. It's hard to debug (which part failed?), difficult to maintain, and often produces unreliable results.

### How we are solving the problem - "A team of specialists"

Instead of one "do-it-all" agent, we can build a multi-agent system. This is a team of simple, specialized agents that collaborate, just like a real-world team. Each agent has one clear job (e.g., one agent only does research, another only writes). This makes them easier to build, easier to test, and much more powerful and reliable when working together.

### Architecture

<img src = "/Users/mario/Desktop/kaggle/Day-1-B/1.png"/>
<img src = "/Users/mario/Desktop/kaggle/Day-1-B/2.png"/>




# 2.1 Research Specialization Assistant - Research and Summarization system

A system with two agents:-
1) Research agent - Searches using Google search
2) Summarizer agent - Creates concise summaries from research findings

In [12]:
# research agent 
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""
    You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.
    """,
    tools=[google_search],
    output_key="research_findings" # the result of this agent will be stored in the session state with this key
)

print("research agent created.")

research agent created.


In [14]:
#summarizer agent 
summarizer_agent = Agent(
    name="SummarizerAgent",
    model = Gemini(
        name="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""
    Read the provided research findings: {research_findings}. Create a concise summary as a bulleted list with 3-5 key points.
    """,
    output_key = "final_summary"
)
print("summarizer agent created")

summarizer agent created


In [19]:
# then we bring the agents under a root agent or a coordinator agent
# below we are wrapping the sub agents in "AgentTool" to make them callable 
# for the root agent

In [18]:
try:
    root_agent = Agent(
        name="RootAgent",
        model=Gemini(
            model="gemini-2.5-flash-lite",
            retry_options = retry_config,
        ),
        instruction=""" You are a research coordinator. Your goal is to answer a user's query by orchestrating a workflow
        1. First you must call the `ResearchAgent` tool to find the relevant information on the topic provided by the user.
        2. Next, after receiving the research findings, you must call `SummarizerAgent` to create a concise summary.
        3. Fiannly, present the final summary clearly to the user as your response""",
        tools = [AgentTool(research_agent), AgentTool(summarizer_agent)],
    )
    print("root agent created")
except Exception as e:
    print(f"Error creating the agent {e}")

root agent created


In [21]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(" Who are the members of PinkFloyd, where are the located right now ")


 ### Created new session: debug_session_id

User >  Who are the members of PinkFloyd, where are the located right now 


RootAgent > The surviving members of Pink Floyd are David Gilmour, Nick Mason, and Roger Waters. David Gilmour is located in England and is touring globally in late 2024 for his new solo album, "Luck and Strange." Nick Mason resides in Wiltshire, England, and is actively touring with his band, Nick Mason's Saucerful of Secrets. Roger Waters is an extensively touring musician who released "Dark Side of the Moon Redux" in 2023; his exact current location is not specified.


# Section 3 - sequential workflows - The Assembly Line

## The Problem: Unpredictable Order

The previous multi-agent system worked, but it relied on a detailed instruction prompt to force the LLM to run steps in order. This can be unreliable. A complex LLM might decide to skip a step, run them in the wrong order, or get "stuck," making the process unpredictable.

## The Solution: A Fixed Pipeline

When you need tasks to happen in a guaranteed, specific order, you can use a SequentialAgent. This agent acts like an assembly line, running each sub-agent in the exact order you list them. The output of one agent automatically becomes the input for the next, creating a predictable and reliable workflow.

### Use Sequential when: Order matters, you need a linear pipeline, or each step builds on the previous one.

Architecture: blog creation pipeline

<img src="/Users/mario/Desktop/kaggle/Day-1-B/3.png"/>



# Blog post creation with sequential agent
1) OUtline agent - creates a blog outline for a given topic
2) Writer agent - writes a blog post
3) Editor agent - Edits a blog post draft for clarity and structure

In [29]:
# OUtline agent - creates an outline agent for blog 
try:
    outline_agent = Agent(
        name="OutlineAgent",
        model=Gemini(
            model = "gemini-2.5-flash-lite",
            retry_options = retry_config
        ),
        instruction=
        """
        You are a specialized writer with a well over experience of writing blogs for more than
        20+ years. you are supposed to make blogs in the how Rick talks to morty, it should match the exact tone
        of how Rick responds, the blog should reflect as if it's Rick who's written it keeping these points in mind:
        1) A Catchy Headline
        2) An Introduction hook
        3) 3-5 main sections with 2-3 bullet points for each
        4) A concluding thought
        """,
        output_key="blog_outline", # the result of the agent will be stored in the session state with the key.

    )
    print("outline agent created")
except Exception as e:
    print(f"The agent failed to create due to {e}")

outline agent created


In [30]:
# Writer agent - writes full blog post based on the outline from teh previous agent

writer_agent = Agent(
    name="WriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction=""" Following this outline strictly: {blog_outline}
    write a blog, 300-400 word post with an engaging and informative tone.""",
    output_key = "blog_draft"
)
print("writer agent created")

writer agent created


In [31]:
# Editor agent - edits and polishes the draft from teh writer agent
editor_agent=Agent(
    name="EditorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""
    edit this draft: {blog_draft}
    Your task is to polish the text by fixing any grammatical errors, improving the flow 
    and sentence structure , enhancing the overall clarity and making it close to How Rick actaully speaks,
    you have to make sure to break the 4th wall. 
    """,
    output_key="final_blog"
)
print("editor_agent created")

editor_agent created


In [32]:
# then we bring all the agents under a sequential agent, which runs the agents in 
# order that they are listed:
try:
    root_agent = SequentialAgent(
        name="BlogPipeline",
        sub_agents=[outline_agent, writer_agent, editor_agent],
    )
    print("the pipeline runs successfully")
except Exception as e:
    print(f"The pipeline is not running due to this error {e}")

the pipeline runs successfully


# now time to test the sequential agents

In [33]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(" Write a blog about AC/DC and black sabbath comparing the two and what they each give. ")


 ### Created new session: debug_session_id

User >  Write a blog about AC/DC and black sabbath comparing the two and what they each give. 
OutlineAgent > Alright, listen up, you jabronis! Rick here, and I'm about to drop some sonic knowledge bombs on your puny brains. Today, we're diving headfirst into the primordial ooze of heavy rock, pitting two titans against each other: AC/DC and Black Sabbath. Don't get me wrong, they're both loud enough to shatter galaxies, but they bring their own brand of interdimensional mayhem to the table. So strap in, grab a flask, and let's see who's the real riff master.

## AC/DC vs. Black Sabbath: Which Butt-Rocking Behemoth Reigns Supreme?

You think you know rock and roll? Pffft. You probably still think listening to The Beach Boys is edgy. Well, let me tell you, these two bands are the real deal. They're not here to serenade your grandma; they're here to melt your face off and leave you begging for more. So, what's the difference, you ask? It's lik